In [1]:
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
import pandas as pd
from glob import glob

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

import albumentations as A
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Data Load & DataSet 생성

- get data(data_dir) : 경로입력하면 해당 경로의 폴더내의 이미지 경로를 리스트로 리턴
- load_img_numpy(img_dir, mask_dir, target_size) : 이미지 경로, 마스크 경로, target_size(resize할 사이즈)를 입력하면 지정한 사이즈의 이미지와 마스크를 numpy array로 리턴
- create dataset(img_dir, mask_dir, batch_size) : 이미지폴더경로, 마스크폴더경로, 배치처리를 한다면 배치 사이즈 입력하면 dataset을 생성하여 리턴

In [2]:
def get_data(data_dir):
    # get image path
    img_path = os.listdir(data_dir)
    try :
        img_path.remove('.ipynb_checkpoints')
    except:
        pass
    img_path_list = [os.path.join(data_dir, x) for x in img_path]
    img_path_list = sorted(img_path_list)
    return img_path_list  # img 들의 경로를 리스트로 리턴

def load_img_numpy(img_dir, mask_dir, target_size) :
    img = cv2.imread(img_dir)
    img = cv2.resize(img, dsize=target_size)
    img = img.astype(np.float32) / 255.0 # normalize
    
    mask = cv2.imread(mask_dir)
    mask = cv2.resize(mask, dsize=target_size)
    mask = mask[..., :1]
    mask = mask.astype(np.float32)
    
    return img, mask

In [3]:
def create_dataset(img_dir, mask_dir, target_size, batch_size=None) :
    x_img_dir = get_data(img_dir)
    y_img_dir = get_data(mask_dir)
    
    x_img = [] # 원본이미지
    y_img = [] # 마스크 이미지
    
    ## 최적화가 필요한 부분
    for i, m in zip(x_img_dir, y_img_dir):
        img, mask = load_img_numpy(i, m, target_size)
        x_img.append(img)
        y_img.append(mask)
        
    x = Dataset.from_tensor_slices(x_img)
    y = Dataset.from_tensor_slices(y_img)
    dataset = Dataset.zip((x, y))
    
    if batch_size is not None:
        dataset = dataset.batch(batch_size, drop_remainder=True).repeat()
   
    return dataset

In [4]:
##### 경로 설정 ######
DATA_DIR = './data/segmentation'

x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'trainannot')

x_valid_dir = os.path.join(DATA_DIR, 'valid')
y_valid_dir = os.path.join(DATA_DIR, 'validannot')

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'testannot')

In [5]:
IMAGE_SIZE = 256
BATCH_SIZE = 4

train_dataset = create_dataset(x_train_dir, y_train_dir, (IMAGE_SIZE,IMAGE_SIZE), BATCH_SIZE)
print('train dataset has made!')
valid_dataset = create_dataset(x_valid_dir, y_valid_dir, (IMAGE_SIZE,IMAGE_SIZE), BATCH_SIZE)
print('valid dataset has made!')

2022-05-04 15:08:11.321230: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-04 15:08:11.321510: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 15:08:11.321750: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


train dataset has made!
valid dataset has made!


In [6]:
train_dataset

<RepeatDataset shapes: ((4, 256, 256, 3), (4, 256, 256, 1)), types: (tf.float32, tf.float32)>

In [7]:
valid_dataset

<RepeatDataset shapes: ((4, 256, 256, 3), (4, 256, 256, 1)), types: (tf.float32, tf.float32)>

((img, mask)) : ((batch_size, width, heigth, channels), (batch_size, width, height, channels))

---
# Model : U-net

- [출처](https://idiotdeveloper.com/unet-implementation-in-tensorflow-using-keras-api/)

In [8]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [9]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [10]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
EPOCHS = 20
STEPS_PER_EPOCH = len(get_data(x_train_dir)) // BATCH_SIZE # 228//4
VAL_STEPS = len(get_data(y_train_dir)) // BATCH_SIZE

In [ ]:
model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_data=valid_dataset,
                          validation_steps = VAL_STEPS)

Epoch 1/20


2022-05-04 15:09:01.227200: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-04 15:09:01.252909: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz


57/57 [==============================] - 721s 13s/step - loss: 0.7020 - accuracy: 0.6813 - val_loss: 0.6036 - val_accuracy: 0.7594
Epoch 2/20
57/57 [==============================] - 715s 13s/step - loss: 0.2464 - accuracy: 0.9654 - val_loss: 0.5380 - val_accuracy: 0.7779
Epoch 3/20
57/57 [==============================] - 715s 13s/step - loss: 0.1902 - accuracy: 0.9777 - val_loss: 0.5285 - val_accuracy: 0.7791
Epoch 4/20
57/57 [==============================] - 716s 13s/step - loss: 0.1614 - accuracy: 0.9820 - val_loss: 0.5355 - val_accuracy: 0.7778
Epoch 5/20
57/57 [==============================] - 714s 13s/step - loss: 0.1634 - accuracy: 0.9809 - val_loss: 0.5625 - val_accuracy: 0.7753
Epoch 6/20
57/57 [==============================] - 673s 12s/step - loss: 0.1356 - accuracy: 0.9842 - val_loss: 0.5629 - val_accuracy: 0.7783
Epoch 7/20
57/57 [==============================] - 671s 12s/step - loss: 0.1249 - accuracy: 0.9849 - val_loss: 0.6033 - val_accuracy: 0.7786
Epoch 8/20
57/57 

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
model.save('./Model_Save/segmentation_epoch20_with_valid.h5')

## Augmentation 추가해야될 것 같습니다